This notebook is intended to show the various functions and functionality of the `WileyCorpusGenerator()` class. The general workflow is stated in the top-level README.md, but the more specific workflow for this publisher is as follows:

1. Use search terms to query the CrossRef Clickthrough API for specific article DOI and API-specific URL's

2. Use the identified URL's, which are API queries, to receive the PDF files of the original articles

3. 

In [ ]:
import json
import os
import sys

module_path = os.path.abspath(os.path.join('../modules/wiley_corpus_generator/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import wiley_corpus_generator as wcg